In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from scipy import stats
from keras import backend as K
from keras.callbacks import TensorBoard, EarlyStopping
from sklearn.model_selection import StratifiedKFold
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import TimeDistributed
from keras.layers import Activation
from keras.layers import MaxPooling1D, UpSampling1D
from keras.layers import MaxPooling2D
from keras.layers import Flatten, Reshape
from keras.layers import Input, Dense, Bidirectional
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import Conv2D
from keras.layers import RepeatVector
from keras.layers.pooling import GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras.utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Model
from keras.models import Sequential
import numpy as np
import os
import matplotlib.pyplot as plt
import random as rand
import pandas as pd
import tensorflow
import keras
import statistics as st
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils.vis_utils import plot_model
from sklearn.preprocessing import LabelEncoder
from collections import Counter

In [ ]:
#os.chdir("/Users/devkash/Desktop/Baboons/Final_Data/")

In [ ]:
import p_f1_recall as metric

results = []

In [ ]:
### HYPERPARAMETERS ###


epochs = 100
batch_size = 200
verbose = 1


group_size = 60
n_sub_group_timesteps = 4
num_filters = 128
LSTM_nodes = 200
encoding_dim = 32
BiD = False
kernel_size = 2



In [ ]:
##### First proces unlabled data

data_u = pd.read_csv('fullest_unlabeled_data.csv')

### Remove unncessary columns
data_u = data_u.iloc[:,[3,4,5,7,8,9,10]]

### Standardize the features
def standardize(data,col_nums_to_standardize):
    for col in col_nums_to_standardize:
        data.iloc[:,col] = (data.iloc[:,col] - np.mean(data.iloc[:,col]))/np.std(data.iloc[:,col])
    return data

data_u = standardize(data_u,col_nums_to_standardize = range(6))


#### Now process the labeled data
data_train = pd.read_csv('final_training.csv')
data_test = pd.read_csv('final_testing.csv')

### Remove unncessary columns
data_train = data_train.iloc[:,[3,4,5,7,8,9,10,12]]  ## Add 13 if need to keep IDs ## Drop heading because of low mutual information
data_test = data_test.iloc[:,[3,4,5,7,8,9,10,12]]

### Standardize the features
data_train = standardize(data_train,col_nums_to_standardize = range(6))
data_test = standardize(data_test,col_nums_to_standardize = range(6))

In [ ]:
#Break features into groups of size n

def break_into_samples(data,group_size):
    l = []
    sep_ts = set(data.loc[:,'ID'])
    for ts in sep_ts:
        sub_data  = np.array(data.loc[data.loc[:,'ID']== ts,:])
        a = int(sub_data.shape[0]/group_size)
        if a != 0:
            sub_data = sub_data[:(a*group_size),:]
            sub_data = [np.split(sub_data,a)]
            l = l + sub_data
    data = np.vstack(l)
    data = data[:,:,:-1]
    return data

chunked_data_train = break_into_samples(data_train,group_size)
chunked_data_test = break_into_samples(data_test,group_size)
chunked_u_data = break_into_samples(data_u,group_size)

In [ ]:
def split_train_val(data,labeled = True,thresh = 0.8):
    np.random.shuffle(data)
    thresh1 = int(data.shape[0]*thresh)
    training = data[:thresh1,:,:]
    val = data[thresh1:,:,:]
    return training, val


global_num_classes = -1

def split_feat_lab(data,many_to_one = True, prop = 0.65):
    features = data[:,:,:data.shape[2]-1]
    labels = data[:,:,data.shape[2]-1]
    if many_to_one == True:
        y_temp = np.zeros(labels.shape[0])
        y_temp.fill(np.nan)
        for i in range(labels.shape[0]):
            row = labels[i,:]
            trans_class = (Counter(row)).most_common(1)[0]
            if trans_class[1] < group_size*prop : 
                y_temp[i] = row[1]
            else:
                 y_temp[i] = st.mode(row)
        labels = y_temp
        labels = to_categorical(labels,num_classes)
        global global_num_classes
    if global_num_classes < 0:
        global_num_classes = len(set(labels.flatten('C')))
    num_classes = global_num_classes
    return features,labels

In [ ]:
chunked_data_train1, chunked_data_val = split_train_val(chunked_data_train)
training1, val1 = split_train_val(chunked_u_data)

train_x1,train_y1 = split_feat_lab(chunked_data_train1,many_to_one = False)
val_x1,val_y1 = split_feat_lab(chunked_data_val,many_to_one = False)
test_x1,test_y1 = split_feat_lab(chunked_data_test,many_to_one = False)

In [ ]:
n_timesteps, n_features = training1.shape[1], training1.shape[2]
n_sub_groups = int(n_timesteps/n_sub_group_timesteps)


def split_samples_into_sub_samples(features,labels,many_to_one=True,prop=0.65):
    features = features.reshape((features.shape[0],n_sub_groups,n_sub_group_timesteps,n_features))
    labels = labels
    if many_to_one == False:
        y_temp = np.zeros((labels.shape[0],n_sub_groups,1))
        y_temp.fill(np.nan)
        labels = labels.reshape(labels.shape[0],n_sub_groups,n_sub_group_timesteps)
        for i in range(labels.shape[0]):
            chunk = labels[i,:,:]
            for j in range(chunk.shape[0]):
                row = chunk[j,:]
                trans_class = (Counter(row)).most_common(1)[0]
                if trans_class[1] < group_size*0.51:
                    y_temp[i,j,0] = row[1]
                else:
                    y_temp[i,j,0] = st.mode(row)
        labels = y_temp
        labels = to_categorical(labels)
    return features,labels

def split_samples_into_sub_samples_u(features):
    features = features.reshape((features.shape[0],n_sub_groups,n_sub_group_timesteps,n_features))
    return features


train_x, train_y = split_samples_into_sub_samples(train_x1, train_y1,many_to_one=False)
val_x, val_y = split_samples_into_sub_samples(val_x1, val_y1,many_to_one=False)
test_x, test_y = split_samples_into_sub_samples(test_x1, test_y1,many_to_one=False)
training = split_samples_into_sub_samples_u(training1)
val = split_samples_into_sub_samples_u(val1)

### if you get an error here, try rerunning the cell above and then running this one

In [ ]:
kernel_size = 2
def build_full_autoencoder(many_to_one = True, bi_directional = BiD):
    raw_input = Input(shape=(n_sub_groups,n_sub_group_timesteps,n_features))
    x = TimeDistributed(Conv1D(filters=num_filters, kernel_size=kernel_size, activation="relu",padding='same'),input_shape=(n_sub_groups,n_sub_group_timesteps,n_features))(raw_input)
    x = TimeDistributed(Conv1D(filters=num_filters, kernel_size=kernel_size, activation = "relu",padding='same'))(x)
    x = TimeDistributed(Dropout(0.5))(x)
    x = TimeDistributed(MaxPooling1D(pool_size=2))(x)
    temp_model = Model(raw_input,x)
    shape2 = temp_model.layers[-1].output_shape[1:]
    x = TimeDistributed(Flatten())(x)
    temp_model = Model(raw_input,x)
    shape1=temp_model.layers[-1].output_shape[1:]
    if many_to_one == False:
        if bi_directional == True:
            x=Bidirectional(LSTM(LSTM_nodes,return_sequences=True))(x)
        else:
            x=LSTM(LSTM_nodes,return_sequences=True)(x)
        x=Dropout(0.5)(x)
        x=TimeDistributed(Dense(encoding_dim*2, activation='relu'))(x)
        encoded=TimeDistributed(Dense(encoding_dim))(x)
        encoder = Model(raw_input,encoded)
        x=TimeDistributed(Dense(encoding_dim*2, activation='relu'))(encoded)
        x=Dropout(0.5)(x)
    else:
        if bi_directional == True:
            x=Bidirectional(LSTM(LSTM_nodes,return_sequences=True))(x)
        else:
            x=LSTM(LSTM_nodes,return_sequences=True)(x)
        x=Dropout(0.5)(x)
        x=TimeDistributed(Dense(encoding_dim*2, activation='relu'))(x)
        encoded=TimeDistributed(Dense(encoding_dim))(x)
        encoder = Model(raw_input,encoded)
        x=TimeDistributed(Dense(encoding_dim*2, activation='relu'))(encoded)
        x=Dropout(0.5)(x)
        x=RepeatVector(n_sub_groups)
    if bi_directional == True:
        x=Bidirectional(LSTM(LSTM_nodes,return_sequences=True))(x)
    else:
        x=LSTM(LSTM_nodes,return_sequences=True)(x)
    x=TimeDistributed(Dense(shape1[1]))(x)
    x=Reshape(shape2)(x)
    x=TimeDistributed(UpSampling1D(2))(x)
    x=TimeDistributed(Conv1D(filters=num_filters, kernel_size=kernel_size, activation = "relu",padding='same'))(x)
    x=TimeDistributed(Conv1D(filters=n_features, kernel_size=kernel_size, activation = "relu",padding='same'))(x)
    autoencoder = Model(raw_input,x)
    return autoencoder,encoder


autoencoder,encoder = build_full_autoencoder(many_to_one = False, bi_directional = True)
autoencoder.summary()
encoder.summary()

init_weights = encoder.get_weights()

def full_auto_train(autoencoder):
    autoencoder.compile(optimizer='adam', loss='mse')
    autoencoder.fit(training, training,
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(val, val),
                callbacks=[EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto', baseline=None, restore_best_weights=True)])
    loss = autoencoder.evaluate(val, val, batch_size=batch_size, verbose=verbose)
    return loss

autoencoder_loss = full_auto_train(autoencoder)

trained_weights = encoder.get_weights()

#autoencoder_2 = build_full_autoencoder(many_to_one=False)
#autoencoder_2.summary()
#full_auto_train(autoencoder_2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_32 (InputLayer)        (None, 15, 4, 6)          0         
_________________________________________________________________
time_distributed_415 (TimeDi (None, 15, 4, 128)        1664      
_________________________________________________________________
time_distributed_416 (TimeDi (None, 15, 4, 128)        32896     
_________________________________________________________________
time_distributed_417 (TimeDi (None, 15, 4, 128)        0         
_________________________________________________________________
time_distributed_418 (TimeDi (None, 15, 2, 128)        0         
_________________________________________________________________
time_distributed_419 (TimeDi (None, 15, 256)           0         
_________________________________________________________________
bidirectional_61 (Bidirectio (None, 15, 400)           731200    
__________

In [ ]:
encoder.set_weights(trained_weights)
## If many-to-one is true:
#new_layer = Dense(global_num_classes, activation='softmax')(encoder.output)
# otherwise, use this one:
new_layer = TimeDistributed(Dense(global_num_classes, activation='softmax'))(encoder.output)
pre_trained = Model(encoder.input,new_layer)
pre_trained.summary()

def train_model(model):
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',metric.recall,metric.precision,metric.f1])
    model.fit(train_x,train_y,epochs=epochs,batch_size=batch_size,verbose=verbose,validation_data=(val_x,val_y),
             callbacks=[EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto', baseline=None, restore_best_weights=True), 
                       TensorBoard(log_dir='./tb')])
    loss,accuracy,recall_score,precision_score,f_score = model.evaluate(test_x, test_y, batch_size=batch_size, verbose=verbose)
    return loss,accuracy,recall_score,precision_score,f_score

m1_loss,m1_accuracy,m1_recall_score,m1_precision_score,m1_f_score = train_model(pre_trained)

In [ ]:
#encoder.set_weights(init_weights)
## If many-to-one is true:
#new_layer = Dense(global_num_classes, activation='softmax')(encoder.output)
# otherwise, use this one:
#new_layer = TimeDistributed(Dense(global_num_classes, activation='softmax'))(encoder.output)
#no_pre_training = Model(encoder.input,new_layer)
#no_pre_training.summary()

#m2_loss,m2_accuracy,m2_recall_score,m2_precision_score,m2_f_score = train_model(no_pre_training)


In [ ]:
results.append([m1_f_score,m1_accuracy,m1_recall_score,m1_precision_score])
import os
os.system('say "done"')


In [ ]:
print(np.mean(results,axis=0))
print(np.std(results,axis=0))